## Create the SQLite Database


In [ ]:
import sqlite3

# Create the database file in your project folder
conn = sqlite3.connect('client_program.db')
cur = conn.cursor()

print("Database created successfully.")

## Create All 4 Tables

In [6]:
# Create Clients table
cur.execute('''
CREATE TABLE IF NOT EXISTS Clients (
    ClientID INTEGER PRIMARY KEY,
    FirstName TEXT,
    LastName TEXT,
    Email TEXT,
    Phone TEXT
)
''')

# Create Programs table
cur.execute('''
CREATE TABLE IF NOT EXISTS Programs (
    ProgramID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    StartDate TEXT,
    EndDate TEXT
)
''')

# Create Enrollments table
cur.execute('''
CREATE TABLE IF NOT EXISTS Enrollments (
    EnrollmentID INTEGER PRIMARY KEY,
    ClientID INTEGER,
    ProgramID INTEGER,
    EnrollmentDate TEXT,
    FOREIGN KEY(ClientID) REFERENCES Clients(ClientID),
    FOREIGN KEY(ProgramID) REFERENCES Programs(ProgramID)
)
''')

# Create Case Notes table
cur.execute('''
CREATE TABLE IF NOT EXISTS Case_Notes (
    NoteID INTEGER PRIMARY KEY,
    ClientID INTEGER,
    NoteDate TEXT,
    Staff TEXT,
    Note TEXT,
    FOREIGN KEY(ClientID) REFERENCES Clients(ClientID)
)
''')

conn.commit()
print("All tables created.")

All tables created.


## Insert Sample Data into Tables

In [8]:
# Insert into Clients
clients = [
    (1, "John", "Doe", "john@example.com", "555-1234"),
    (2, "Jane", "Smith", "jane@example.com", "555-5678"),
    (3, "Robert", "Brown", "robert@example.com", "555-9101"),
]

cur.executemany("INSERT INTO Clients VALUES (?, ?, ?, ?, ?)", clients)

# Insert into Programs
programs = [
    (1, "Housing Assistance", "Support", "2024-01-01", "2024-12-31"),
    (2, "Job Training", "Education", "2024-02-01", "2024-08-01"),
    (3, "Health Counseling", "Wellness", "2024-03-15", "2024-10-15"),
]

cur.executemany("INSERT INTO Programs VALUES (?, ?, ?, ?, ?)", programs)

# Insert into Enrollments
enrollments = [
    (1, 1, 1, "2024-01-15"),
    (2, 1, 2, "2024-02-15"),
    (3, 2, 1, "2024-02-20"),
    (4, 3, 3, "2024-04-10"),
]

cur.executemany("INSERT INTO Enrollments VALUES (?, ?, ?, ?)", enrollments)

# Insert into Case_Notes
case_notes = [
    (1, 1, "2024-01-16", "Emily Lee", "Intake completed"),
    (2, 2, "2024-02-21", "Emily Lee", "Referral to housing"),
    (3, 3, "2024-04-11", "Michael Chen", "Counseling session logged"),
]

cur.executemany("INSERT INTO Case_Notes VALUES (?, ?, ?, ?, ?)", case_notes)

conn.commit()
print("Sample data inserted.")


Sample data inserted.


## Test If It Works

In [10]:
import pandas as pd

# View Clients table
pd.read_sql("SELECT * FROM Clients", conn)


,ClientID,FirstName,LastName,Email,Phone
0,1,John,Doe,john@example.com,555-1234
1,2,Jane,Smith,jane@example.com,555-5678
2,3,Robert,Brown,robert@example.com,555-9101


In [12]:
pd.read_sql("SELECT * FROM Programs", conn)
pd.read_sql("SELECT * FROM Enrollments", conn)
pd.read_sql("SELECT * FROM Case_Notes", conn)


,NoteID,ClientID,NoteDate,Staff,Note
0,1,1,2024-01-16,Emily Lee,Intake completed
1,2,2,2024-02-21,Emily Lee,Referral to housing
2,3,3,2024-04-11,Michael Chen,Counseling session logged


## Create a SQL View for Easy Analysis

In [25]:
# Create a unified view combining Clients, Programs, Enrollments, and Notes
query_view = '''
CREATE VIEW IF NOT EXISTS ClientProgramView AS
SELECT 
  c.FirstName || ' ' || c.LastName AS ClientName,
  p.Name AS ProgramName,
  p.Type,
  e.EnrollmentDate,
  n.NoteDate,
  n.Staff,
  n.Note
FROM Clients c
JOIN Enrollments e ON c.ClientID = e.ClientID
JOIN Programs p ON e.ProgramID = p.ProgramID
LEFT JOIN Case_Notes n ON c.ClientID = n.ClientID;
'''

cur.execute(query_view)
conn.commit()
print("View created successfully.")


View created successfully.


## Query Clients Enrolled in 2024 Only

In [28]:
query_2024 = '''
SELECT *
FROM ClientProgramView
WHERE EnrollmentDate BETWEEN '2024-01-01' AND '2024-12-31'
ORDER BY EnrollmentDate;
'''
pd.read_sql(query_2024, conn)


,ClientName,ProgramName,Type,EnrollmentDate,NoteDate,Staff,Note
0,John Doe,Housing Assistance,Support,2024-01-15,2024-01-16,Emily Lee,Intake completed
1,John Doe,Job Training,Education,2024-02-15,2024-01-16,Emily Lee,Intake completed
2,Jane Smith,Housing Assistance,Support,2024-02-20,2024-02-21,Emily Lee,Referral to housing
3,Robert Brown,Health Counseling,Wellness,2024-04-10,2024-04-11,Michael Chen,Counseling session logged


## Programs with > 1 Note Per Client (JOIN + GROUP BY + HAVING)

In [31]:
query_notes = '''
SELECT p.Name, COUNT(n.NoteID) * 1.0 / COUNT(DISTINCT c.ClientID) AS NotesPerClient
FROM Programs p
JOIN Enrollments e ON p.ProgramID = e.ProgramID
JOIN Clients c ON c.ClientID = e.ClientID
JOIN Case_Notes n ON n.ClientID = c.ClientID
GROUP BY p.Name
HAVING NotesPerClient > 1;
'''
pd.read_sql(query_notes, conn)


,Name,NotesPerClient


## Clients With Below-Average Program Count (SUBQUERY)

In [34]:
query_sub = '''
SELECT ClientID, COUNT(*) AS ProgramCount
FROM Enrollments
GROUP BY ClientID
HAVING ProgramCount < (
  SELECT AVG(pc) FROM (
    SELECT COUNT(*) AS pc FROM Enrollments GROUP BY ClientID
  )
);
'''
pd.read_sql(query_sub, conn)


,ClientID,ProgramCount
0,2,1
1,3,1


## Export Notes by Staff to Excel (Dynamic Filter)

In [37]:
staff_name = 'Emily Lee'

df_notes = pd.read_sql(f'''
SELECT * FROM Case_Notes
WHERE Staff = "{staff_name}"
''', conn)

output_path = r'C:\Users\ysaya\Desktop\Projects\ASU\IFT 200\Notes_By_Emily.xlsx'
df_notes.to_excel(output_path, index=False)

print("Exported notes to Excel.")


Exported notes to Excel.
